In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver import Firefox, Chrome
import pandas as pd
import re
from bs4 import BeautifulSoup as bs
import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...


True

In [8]:
filename = 'amazon_urls.txt'
file = open('amazon_urls.txt')

In [9]:
LIST=re.split("\r\n|\n",file.read())
file.close()

In [10]:
x = []
for i in LIST:
    print(i.split('~')[-1])
    x.append(i.split('~')[-1])

https://www.amazon.com/Intel-i5-12600K-Desktop-Processor-Unlocked/dp/B09FX4D72T
https://www.amazon.com/Intel-i7-12700KF-Desktop-Processor-Unlocked/dp/B09FXKHN7M


In [11]:
x

['https://www.amazon.com/Intel-i5-12600K-Desktop-Processor-Unlocked/dp/B09FX4D72T',
 'https://www.amazon.com/Intel-i7-12700KF-Desktop-Processor-Unlocked/dp/B09FXKHN7M']

In [12]:
filename.split(".")

['amazon_urls', 'txt']

In [13]:
len(x)

2

In [14]:
x

['https://www.amazon.com/Intel-i5-12600K-Desktop-Processor-Unlocked/dp/B09FX4D72T',
 'https://www.amazon.com/Intel-i7-12700KF-Desktop-Processor-Unlocked/dp/B09FXKHN7M']

In [15]:
reviews,reviewedon,reviewerplace,reviewratings, prodname, dfs = [],[],[],[],[],[]
driver = Firefox()
df = pd.DataFrame()
driver.maximize_window()
k = 0
print(x)
while k != len(x):
    
    # print(k)
    url = x[k]
    try:
        driver.get(url)
        product_name = url.split("/")[3].replace("-"," ")
        print(product_name, end='\n')
        prodname.append(product_name)
    except Exception as ex:
        print(ex)
    
        
    try:
        all_reviews = driver.find_element(By.CSS_SELECTOR,'#reviews-medley-footer .a-text-bold')
        all_reviews.click()
    except Exception as ex:
        print(ex)

    
    current_page=0
    page_limit=3
    while True:
        # Extract page number from the URL
        page_number = re.search(r'pageNumber=(\d+)', driver.current_url)
        if page_number:
            current_page = int(page_number.group(1))
        else:
            current_page = 1
        
        print(f"Scraping page {current_page}")
        
        ratings = driver.find_elements(By.XPATH, '//i[@data-hook="review-star-rating"]/span')
        loc_date = driver.find_elements(By.XPATH, "//span[@data-hook='review-date']")
        review = driver.find_elements(By.XPATH, '//span[@data-hook="review-body"]')
        
        try:
            next_btn = driver.find_element(By.CSS_SELECTOR, "#cm_cr-pagination_bar a")
        except:
            pass
        for i in range(len(review)):
            try:
                other, date = loc_date[i].text.split('on')
            except:
                pass
            place = other.split(' ')[3] +" "+ other.split(" ")[4]
            reviewedon.append(date)
            reviewerplace.append(place)
   
            try:
                rating = ratings[i].get_attribute('textContent')
            except:
                rating = 'Not Available'
            reviewratings.append(rating)
            try:
                reviews.append(review[i].text)
                # print(review[i].text)
            except:
                pass
   
        try:
            next_btn = driver.find_element(By.XPATH, "//ul[@class='a-pagination']/li[2]/a")
            if next_btn and 'a-disabled' in next_btn.get_attribute("class"):
                print("No more pages. Exiting loop.")
                break
            else:
                next_btn.click()
                time.sleep(5)
        except NoSuchElementException:
            print("No 'next' button found. Exiting loop.")
            # print("Current URL:", driver.current_url)
            break
        except Exception as ex:
            print("An error occurred:", ex)
            break
        if current_page==page_limit:
            break
    #Data Pre-processing & Cleaning
    reviewratings=[str(reviewrating).replace(' out of 5 stars', '') for reviewrating in reviewratings]
    print(reviewratings)
    
    # Removing Punctuations & Converting to Lowercase
    for i in range(len(reviews)):
        reviews[i] = re.sub(r'[^A-Za-z\s]', '', reviews[i])  
        reviews[i] = reviews[i].lower()
    
    print(reviews)

    #Removing Custom Stop Words
    custom_words = [
    "against", "above", "below", "out", "under", "again",
    "no", "nor", "not", "don't", "should", "didn't",
    "does", "doesn't", "hadn't", "hasn't", "haven't",
    "isn't", "mustn", "mustn't", "should", "shouldn't",
    "was", "wasn't", "weren", "weren't", "won't", "wouldn't"
     ]

    stop_words = set(stopwords.words('english'))
    stop_words.update(custom_words)

    for i in range(len(reviews)):
        words = re.findall(r'\b\w+\b', reviews[i])
    
        filtered_words = [word for word in words if word.lower() not in stop_words]
    
        filtered_review = ' '.join(filtered_words)
    
        reviews[i] = filtered_review


    

    # Lemmetization
    lemmatizer = WordNetLemmatizer()
    for i in range(len(reviews)):
        words = word_tokenize(reviews[i])
        lemmatized_review = ' '.join([lemmatizer.lemmatize(word) for word in words])
        reviews[i] = lemmatized_review

    
    data = {'Reviews':reviews, 'Date':reviewedon,'Place':reviewerplace,
       'Rating':reviewratings}
    min_length = len(reviews)
    df = pd.DataFrame({k:pd.Series(v[:min_length]) for k,v in data.items()})
    df['Product'] = product_name
    dfs.append(df)
    del df
    k+=1
    
finaldata = pd.concat(dfs,axis=0)
finaldata.to_csv('../data/Amazon_data.csv',index=False)
driver.close()
print("Finished")

The geckodriver version (0.31.0) detected in PATH at c:\Users\KIIT\OneDrive\codes\vscodes\projects\intel\scraper\final\amazon\geckodriver.exe might not be compatible with the detected firefox version (126.0.0.8895); currently, geckodriver 0.34.0 is recommended for firefox 126.*, so it is advised to delete the driver in PATH and retry


['https://www.amazon.com/Intel-i5-12600K-Desktop-Processor-Unlocked/dp/B09FX4D72T', 'https://www.amazon.com/Intel-i7-12700KF-Desktop-Processor-Unlocked/dp/B09FXKHN7M']
Intel i5 12600K Desktop Processor Unlocked
Scraping page 1
Scraping page 1
No 'next' button found. Exiting loop.
['5.0', '5.0', '5.0', '5.0', '5.0', '5.0', '5.0', '5.0', '5.0', '5.0']
['nice cpu this replaces my old k pc and what an amazing difference and a pleasant surprise performancewise it runs cool  degrees during stress tests and gaming and it is stable to  ghz havent tried to push it but have little doubt there is sufficient headroom to oc further this has turned out to be an outstanding cpu i recommend this cpu to anyone interested in a lowcost lowheat and stable performance', 'ble operation first class product', 'the k cpu has been an outstanding upgrade from my previous setup delivering a remarkable boost in performance ive been particularly impressed by its cool operation maintaining a stable  degrees even dur